In [1]:
!pip install transformers
!pip install transformers datasets
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 7.6 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import numpy as np
raw_datasets=load_dataset('squad')
raw_datasets

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [3]:
raw_datasets['train'][1]

{'id': '5733be284776f4190066117f',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'What is in front of the Notre Dame Main Building?',
 'answers': {'text': ['a copper statue of Christ'], 'answer_start': [188]}}

In [4]:
from transformers import AutoTokenizer
model_checkpoint='distilbert-base-cased'
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
context=raw_datasets['train'][1]['context']
question=raw_datasets['train'][1]['question']
inputs=tokenizer(question,context)
print(inputs)
print(tokenizer.decode(inputs['input_ids']))

{'input_ids': [101, 1327, 1110, 1107, 1524, 1104, 1103, 10360, 8022, 4304, 4334, 136, 102, 22182, 1193, 117, 1103, 1278, 1144, 170, 2336, 1959, 119, 1335, 4184, 1103, 4304, 4334, 112, 188, 2284, 10945, 1110, 170, 5404, 5921, 1104, 1103, 6567, 2090, 119, 13301, 1107, 1524, 1104, 1103, 4304, 4334, 1105, 4749, 1122, 117, 1110, 170, 7335, 5921, 1104, 4028, 1114, 1739, 1146, 14089, 5591, 1114, 1103, 7051, 107, 159, 21462, 1566, 24930, 2508, 152, 1306, 3965, 107, 119, 5893, 1106, 1103, 4304, 4334, 1110, 1103, 19349, 1104, 1103, 11373, 4641, 119, 13301, 1481, 1103, 171, 17506, 9538, 1110, 1103, 144, 10595, 2430, 117, 170, 14789, 1282, 1104, 8070, 1105, 9284, 119, 1135, 1110, 170, 16498, 1104, 1103, 176, 10595, 2430, 1120, 10111, 20500, 117, 1699, 1187, 1103, 6567, 2090, 25153, 1193, 1691, 1106, 2216, 17666, 6397, 3786, 1573, 25422, 13149, 1107, 8109, 119, 1335, 1103, 1322, 1104, 1103, 1514, 2797, 113, 1105, 1107, 170, 2904, 1413, 1115, 8200, 1194, 124, 11739, 1105, 1103, 3487, 17917, 114, 117

In [6]:
#what if the context is very long
#split it into multiple samples
inputs=tokenizer(

                 question,
                 context,
                 max_length=100,
                 truncation='only_second',
                 stride=50,
                 return_overflowing_tokens=True
)
for ids in inputs['input_ids']:
  print(tokenizer.decode(ids))


[CLS] What is in front of the Notre Dame Main Building? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the G [SEP]
[CLS] What is in front of the Notre Dame Main Building? [SEP] facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernade [SEP]
[CLS] What is in front of the Notre Dame Main Building? [SEP] of the Sacred Heart. Immediately behind the basilica is the Grotto, 

In [7]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping'])

In [8]:
#what's the new key?
inputs['overflow_to_sample_mapping']

[0, 0, 0, 0]

In [9]:
inputs=tokenizer(
    raw_datasets['train'][:3]['question'],
    raw_datasets['train'][:3]['context'],
    max_length=100,
    truncation='only_second',
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True
)
inputs['overflow_to_sample_mapping']

[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2]

In [10]:
for ids in inputs['input_ids']:
  print(tokenizer.decode(ids))

[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basi [SEP]
[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin [SEP]
[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Next to the Main Building is the B

In [11]:
#recreate inputs for single context-question pair
inputs=tokenizer(
    question,
    context,
    max_length=100,
    truncation='only_second',
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True
)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [12]:
#what is offset mapping?
#it tells us the location of tokens.
#note:special tokens takes up 0 space-(0,0)
#the question portion is same for each sample
inputs['offset_mapping']

[[(0, 0),
  (0, 4),
  (5, 7),
  (8, 10),
  (11, 16),
  (17, 19),
  (20, 23),
  (24, 29),
  (30, 34),
  (35, 39),
  (40, 48),
  (48, 49),
  (0, 0),
  (0, 13),
  (13, 15),
  (15, 16),
  (17, 20),
  (21, 27),
  (28, 31),
  (32, 33),
  (34, 42),
  (43, 52),
  (52, 53),
  (54, 56),
  (56, 58),
  (59, 62),
  (63, 67),
  (68, 76),
  (76, 77),
  (77, 78),
  (79, 83),
  (84, 88),
  (89, 91),
  (92, 93),
  (94, 100),
  (101, 107),
  (108, 110),
  (111, 114),
  (115, 121),
  (122, 126),
  (126, 127),
  (128, 139),
  (140, 142),
  (143, 148),
  (149, 151),
  (152, 155),
  (156, 160),
  (161, 169),
  (170, 173),
  (174, 180),
  (181, 183),
  (183, 184),
  (185, 187),
  (188, 189),
  (190, 196),
  (197, 203),
  (204, 206),
  (207, 213),
  (214, 218),
  (219, 223),
  (224, 226),
  (226, 229),
  (229, 232),
  (233, 237),
  (238, 241),
  (242, 248),
  (249, 250),
  (250, 251),
  (251, 254),
  (254, 256),
  (257, 259),
  (260, 262),
  (263, 264),
  (264, 265),
  (265, 268),
  (268, 269),
  (269, 270),
 

In [13]:
len(inputs['offset_mapping'])

4

In [14]:
len(inputs['offset_mapping'][0])

100

In [15]:
print(inputs.sequence_ids(0))

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [16]:
#problem:the position of answer will change in each
#window of the context
#the answer is also the target for the neural network
#how can we recompute the targets for each context window?

#since we took the question and context from the sample earlier
answer=raw_datasets['train'][1]['answers']
answer

{'text': ['a copper statue of Christ'], 'answer_start': [188]}

In [17]:
type(inputs.sequence_ids(0))

list

In [18]:
#find the start and the end of the context(the frist and the last '1')
sequence_ids=inputs.sequence_ids(0)
ctx_start=sequence_ids.index(1)#first occurence
ctx_end=len(sequence_ids)-sequence_ids[::-1].index(1)-1 #last occurence

ctx_start,ctx_end

(13, 98)

In [19]:
#check wheather or not the answer is fully connected within the context
#if not target is (start,end)=(0,0)
ans_start_char=answer['answer_start'][0]
ans_end_char=ans_start_char+len(answer['text'][0])

offset=inputs['offset_mapping'][0]
start_idx=0
end_idx=0

if offset[ctx_start][0]>ans_start_char or offset[ctx_end][1]<ans_end_char:
  print(" target is (0,0)")
  #nothing to do

else:
  #find the start and the end token position
  #the trick is knowing what is in units of tokens and what is in
  #units of characters
  #recall:offset mapping contains the character position of each tokens
  i =ctx_start

  for start_end_char in offset[ctx_start:]:
    start,end=start_end_char

    if start==ans_start_char:
      start_idx=i
      #don't break yet

    if end==ans_end_char:
      end_idx=i
      break

    i += 1

start_idx,end_idx

(53, 57)

In [20]:
#check
input_ids=inputs['input_ids'][0]
input_ids[start_idx:end_idx+1]

[170, 7335, 5921, 1104, 4028]

In [21]:
tokenizer.decode(input_ids[start_idx:end_idx+1])

'a copper statue of Christ'

In [22]:
def find_answer_token_idx(ctx_start,ctx_end,ans_start_char,ans_end_char,offset):
  start_idx=0
  end_idx=0

  if offset[ctx_start][0]>ans_start_char or offset[ctx_end][1] < ans_end_char:
    pass

  else:
    i =ctx_start
    for start_end_char in offset[ctx_start:]:
      start,end=start_end_char

      if start==ans_start_char:
        start_idx=i

      if end==ans_end_char:
        end_idx=i
        break

      i += 1

  return start_idx, end_idx

In [23]:
#try is on all context_windows
#sometimes the answer won't appear

start_idxs=[]
end_idxs=[]

for i ,offset in enumerate(inputs['offset_mapping']):
  #the final window may not be full size -cannot assume 100
  sequence_ids=inputs.sequence_ids(i)

  #find start and end of the context
  ctx_start = sequence_ids.index(1)
  ctx_end = len(sequence_ids)-sequence_ids[::-1].index(1)-1

  start_idx, end_idx = find_answer_token_idx(ctx_start,ctx_end,ans_start_char,ans_end_char,offset)

  start_idxs.append(start_idx)
  end_idxs.append(end_idx)

start_idxs,end_idxs

([53, 17, 0, 0], [57, 21, 0, 0])

In [24]:
#some questions have leading and/or trailing whitespaces
for q in raw_datasets['train']['question'][:1000]:
  if q.strip()!=q:
    print(q)


In what city and state did Beyonce  grow up? 
 The album, Dangerously in Love  achieved what spot on the Billboard Top 100 chart?
Which song did Beyonce sing at the first couple's inaugural ball? 
What event did Beyoncé perform at one month after Obama's inauguration? 
Where was the album released? 
What movie influenced Beyonce towards empowerment themes? 


In [25]:
#now we are ready to process/tokenize the training data
#ie.expand question +context pairs into question+smaller context windows

#google used 384 for squad
max_length=384
stride=128

def tokenize_fn_train(batch):
  #some questions have leading and/or trailing whitespaces
  questions=[q.strip() for q in batch['question']]

  #tokenize the data with padding this time
  #since most contexts are long, we won't bother to pay pad per-mini batch
  inputs=tokenizer(
      questions,
      batch['context'],
      max_length=max_length,
      truncation='only_second',
      stride=stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding='max_length',
  )

  #we don't need these later so remove them
  offset_mapping=inputs.pop('offset_mapping')
  orig_sample_idxs=inputs.pop('overflow_to_sample_mapping')
  answers=batch['answers']
  start_idxs,end_idxs=[],[]

  #same loop as above
  for i ,offset in enumerate(offset_mapping):
    sample_idx=orig_sample_idxs[i]
    answer=answers[sample_idx]

    ans_start_char=answer['answer_start'][0]
    ans_end_char=ans_start_char+len(answer['text'][0])

    sequence_ids=inputs.sequence_ids(i)

    #find start+end of the context
    ctx_start=sequence_ids.index(1)
    ctx_end=len(sequence_ids)-sequence_ids[::-1].index(1)-1

    start_idx, end_idx = find_answer_token_idx(ctx_start,ctx_end,ans_start_char,ans_end_char,offset)

    start_idxs.append(start_idx)
    end_idxs.append(end_idx)

  inputs['start_positions']=start_idxs
  inputs['end_positions']=end_idxs
  return inputs

In [26]:
train_dataset=raw_datasets['train'].map(tokenize_fn_train,
                                        batched=True,
                                        remove_columns=raw_datasets['train'].column_names,)
len(raw_datasets['train']),len(train_dataset)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

(87599, 88729)

In [27]:
#note we will keep thes ids for later
raw_datasets['validation'][0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],


In [28]:
#tokenize the validation set differentley
# we won't need the target since we will just compare with the original answer
#also overrite offset mapping with none  in place of question

def tokenize_fn_validation(batch):
  questions=[q.strip() for q in batch['question']]

  #tokenize the data with padding
  #since most contexts are long we won't bother to pad per-mini batch
  inputs=tokenizer(
      questions,
      batch['context'],
      max_length=max_length,
      truncation='only_second',
      stride=stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding='max_length',
  )

  #we don't need these later so remove them

  orig_sample_idxs=inputs.pop('overflow_to_sample_mapping')
  sample_ids=[]

  #rewrite offset mapping by replacing question typles with none
  #this will be helpfull later when we compute matrics
  for i in range(len(inputs['input_ids'])):
    sample_idx=orig_sample_idxs[i]
    sample_ids.append(batch['id'][sample_idx])

    sequence_ids=inputs.sequence_ids(i)
    offset=inputs['offset_mapping'][i]
    inputs['offset_mapping'][i] = [x if sequence_ids[j]==1 else None for j,x in enumerate(offset)]

  inputs['sample_id']=sample_ids
  return inputs


In [29]:
validation_dataset=raw_datasets['validation'].map(tokenize_fn_validation,
                                                  batched=True,
                                                  remove_columns=raw_datasets['validation'].column_names,)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [30]:
len(raw_datasets['validation']),len(validation_dataset)

(10570, 10822)

In [31]:
from datasets import load_metric
metric=load_metric('squad')


<ipython-input-31-f110cfb6c6c0>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric=load_metric('squad')


In [32]:
predicted_answers = [
    {'id':1,'prediction_text':'Albert Einstein'},
     {'id':2,'prediction_text':'physicist'},
     {'id':3,'prediction_text':'general relativity'}
                   ]
true_answers = [
    {'id':'1', 'answers': {'text': ['Albert Einstein'],'answer_start':[100]}},
    {'id':'2', 'answers': {'text': ['physicist'],'answer_start':[100]}},
    {'id':'3', 'answers': {'text': ['special relaivity'],'answer_start':[100]}}
    ]

    #id and answer start seem superflous but you'll get error if not included
    #
metric.compute(predictions=predicted_answers,references=true_answers)


{'exact_match': 66.66666666666667, 'f1': 66.66666666666667}

In [33]:
#next problem: how to go from logits to prediction text
#to make it easier let's work on an already trained question-answering model
small_validation_dataset=raw_datasets['validation'].select(range(100))
trained_checkpoint='distilbert-base-cased-distilled-squad'
tokenizer2=AutoTokenizer.from_pretrained(trained_checkpoint)

#temporarily assign tokenizer2 to tokenizer since it's used as global
#in tokenize_fn_validation
old_tokenizer=tokenizer
tokenizer=tokenizer2

small_validation_processed=small_validation_dataset.map(
    tokenize_fn_validation,
    batched=True,
    remove_columns=raw_datasets['validation'].column_names
)
#change it back
tokenizer=old_tokenizer

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [34]:
#get the model outputs
import torch
from transformers import AutoModelForQuestionAnswering

#the trained model does not use these columns
small_model_inputs=small_validation_processed.remove_columns(
    ['sample_id','offset_mapping']
)
small_model_inputs.set_format('torch')

#get gpu device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#move tensor to gpu device
small_model_inputs_gpu={
    k:small_model_inputs[k].to(device) for k in small_model_inputs.column_names
}
#download the model
trained_model=AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(device)

#get the model outputs
with torch.no_grad():
  outputs=trained_model(**small_model_inputs_gpu)

In [35]:
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ -2.2607,  -5.1783,  -5.2709,  ...,  -9.5243,  -9.5183,  -9.5288],
        [ -2.5961,  -5.5482,  -5.5313,  ...,  -9.9598,  -9.9533,  -9.9860],
        [ -3.7127,  -7.1848,  -8.5388,  ..., -11.6557, -11.6571, -11.6505],
        ...,
        [ -2.0260,  -4.4167,  -4.4980,  ...,  -8.1479,  -8.1530,  -8.1760],
        [ -4.1553,  -5.8304,  -7.1643,  ..., -10.5255, -10.5251, -10.4890],
        [ -3.2000,  -5.8162,  -6.7249,  ...,  -9.4935,  -9.5038,  -9.4871]]), end_logits=tensor([[ -0.7353,  -4.9236,  -5.1048,  ...,  -8.8734,  -8.8915,  -8.8550],
        [ -1.3056,  -5.3870,  -5.4945,  ...,  -9.4895,  -9.5039,  -9.4959],
        [ -2.7649,  -7.2201,  -9.0916,  ..., -11.3106, -11.3414, -11.2702],
        ...,
        [ -0.0768,  -4.8210,  -4.4374,  ...,  -8.0483,  -8.0502,  -7.9903],
        [ -2.7347,  -5.3650,  -7.2549,  ..., -10.0498, -10.0661,  -9.9886],
        [ -1.0991,  -4.2569,  -6.1267,  ...,  -8.6882,  -8.6889,  -8.627

In [36]:
start_logits=outputs.start_logits.cpu().numpy()
end_logits=outputs.end_logits.cpu().numpy()

In [37]:
small_validation_processed['sample_id'][:5]

['56be4db0acb8001400a502ec',
 '56be4db0acb8001400a502ed',
 '56be4db0acb8001400a502ee',
 '56be4db0acb8001400a502ef',
 '56be4db0acb8001400a502f0']

In [38]:
len(validation_dataset['sample_id'])

10822

In [39]:
len(set(validation_dataset['sample_id']))

10570

In [40]:
#example:{''}
sample_id2idxs={}
for i,id_ in enumerate(small_validation_processed['sample_id']):
  if id_ not in sample_id2idxs:
    sample_id2idxs[id_]=[i]
  else:
    print('here')
    sample_id2idxs[id_].append(i)


start_logits.shape,end_logits.shape


((100, 384), (100, 384))

In [41]:
#reminder of how to find indices with the largest values
(-start_logits[0]).argsort()



array([ 46,  57,  47,  38,  39,  58,  50,  43,  45,  54,  56,  49,  13,
        42,  40,  35,  27,  31,  48,  41,  53,  44,  37,  59,  78,  15,
         0,  52,  24,  65,  81,  70,  18,  51,  55,  26,  69,  29,  28,
        75,  61,  64,  23,  36,  32,  11, 101,  62,  66,  34,  95,  30,
        63,  21,  19,  20,  17,  14,  22,  33,  68,  87, 171,  12,  76,
        71,  73,  92, 110,  84, 151,   1,  74,   2,   6,  16,  80,  79,
       105,  98,  10,  96, 136, 169, 106, 100,  93, 165,  67, 109,   8,
        90,   3, 115,  60,   5,  97,   7, 103, 102,  86,  72, 111,  89,
       108,   4,  88,  25, 132,  77, 123, 150, 124, 153,  83, 118,  82,
        85, 107, 114, 143, 164, 137, 130, 166, 159, 131,  91,   9, 144,
       139, 160,  94, 141, 128, 112, 134, 152, 170, 154, 117, 127, 104,
       140, 157, 155, 133, 145, 119, 162, 138, 135, 156, 167, 168, 126,
       148, 163, 161, 116,  99, 120, 142, 158, 125, 146, 113, 121, 147,
       149, 129, 122, 311, 312, 304, 309, 313, 310, 300, 307, 31

In [42]:
start_logits[0][(-start_logits[0]).argsort()]

array([10.694445  ,  9.803685  ,  4.4599743 ,  4.400488  ,  2.943783  ,
        2.7017365 ,  2.012652  ,  1.578078  ,  0.52237403,  0.02073932,
       -0.02802548, -0.04970503, -0.38572538, -0.69453716, -0.7979469 ,
       -0.8678012 , -0.87220144, -1.3516847 , -1.3703673 , -1.3878812 ,
       -1.5135032 , -1.7355448 , -1.8827012 , -1.8932868 , -1.9078932 ,
       -1.9304957 , -2.2607293 , -2.2983854 , -2.3069293 , -2.502737  ,
       -2.5100586 , -2.530837  , -2.5399904 , -2.6718087 , -2.7323527 ,
       -2.771015  , -2.7713625 , -2.9521286 , -3.0604622 , -3.1706011 ,
       -3.2045465 , -3.5693336 , -3.5798001 , -3.6668804 , -3.7250557 ,
       -3.7498548 , -3.7632139 , -3.9968119 , -4.011324  , -4.0687966 ,
       -4.0944815 , -4.1954722 , -4.238309  , -4.332359  , -4.352411  ,
       -4.3879614 , -4.388608  , -4.3966093 , -4.6790495 , -4.703027  ,
       -4.775753  , -4.777808  , -4.7882147 , -4.7882433 , -4.822122  ,
       -4.8725367 , -4.884931  , -4.8981423 , -5.072093  , -5.10

In [43]:
#reminder:in offset mapping we store None everywhere except the context window
#in context window we store tuple for each token certainity
#(start_character_position,end_character_position)
small_validation_processed['offset_mapping'][0]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 [0, 5],
 [6, 10],
 [11, 13],
 [14, 17],
 [18, 20],
 [21, 29],
 [30, 38],
 [39, 43],
 [44, 46],
 [47, 56],
 [57, 60],
 [61, 69],
 [70, 72],
 [73, 76],
 [77, 85],
 [86, 94],
 [95, 101],
 [102, 103],
 [103, 106],
 [106, 107],
 [108, 111],
 [112, 115],
 [116, 120],
 [121, 127],
 [127, 128],
 [129, 132],
 [133, 141],
 [142, 150],
 [151, 161],
 [162, 163],
 [163, 166],
 [166, 167],
 [168, 176],
 [177, 183],
 [184, 191],
 [192, 200],
 [201, 204],
 [205, 213],
 [214, 222],
 [223, 233],
 [234, 235],
 [235, 238],
 [238, 239],
 [240, 248],
 [249, 257],
 [258, 266],
 [267, 269],
 [269, 270],
 [270, 272],
 [273, 275],
 [276, 280],
 [281, 286],
 [287, 292],
 [293, 298],
 [299, 303],
 [304, 309],
 [309, 310],
 [311, 314],
 [315, 319],
 [320, 323],
 [324, 330],
 [331, 333],
 [334, 342],
 [343, 344],
 [344, 345],
 [346, 350],
 [350, 351],
 [352, 354],
 [355, 359],
 [359, 360],
 [360, 361],
 [362, 369],
 [370, 37

In [44]:
n_largest=20
max_answer_length=30
predicted_answers=[]

#we are looping through the original (untokenized dataset)
#because we need to grab the answer from the original string context
for sample in small_validation_dataset:
  sample_id = sample['id']
  context=sample['context']

  #update these as we loop through candidate answers
  best_score=float('-inf')
  best_answer=None

  #now loop through the expanded input samples (fixed size context windows)
  #from here we will pick the highest probablity start/end combination
  for idx in sample_id2idxs[sample_id]:
    start_logit=start_logits[idx]
    end_logit=end_logits[idx]
    offsets=small_validation_processed[idx]['offset_mapping']

    start_indices=(-start_logit).argsort()
    end_indices=(-end_logit).argsort()

    for start_idx in start_indices[:n_largest]:
      for end_idx in end_indices[:n_largest]:
        #skip answers not contained in context windows
        #recall:we set  entries not pretraining to context None earlier

        if offsets[start_idx] is None or offsets[end_idx] is None:
          continue
        #skip answers where end<start

        if end_idx<start_idx:
          continue
          #skip answers that are too long

        if end_idx-start_idx + 1> max_answer_length:
          continue
        #see theory lecture for more score calculation
        score=start_logit[start_idx] + end_logit[end_idx]

        if score > best_score:
          best_score = score
          #find position of end and start characters
          #recall:ofssets contain tuples for each token
          first_ch=offsets[start_idx][0]
          last_ch=offsets[end_idx][1]

          best_answer=context[first_ch:last_ch]
  #save the best answer
  predicted_answers.append({'id':sample_id,'prediction_text':best_answer})





In [45]:
small_validation_dataset['answers'][0]


{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
 'answer_start': [177, 177, 177]}

In [46]:
#now test it
true_answers=[{'id' : x['id'], 'answers' : x['answers'] } for x in small_validation_dataset]
metric.compute(predictions=predicted_answers,references=true_answers)

{'exact_match': 83.0, 'f1': 88.25000000000004}

In [47]:
from tqdm import tqdm
from collections import defaultdict
import numpy as np

def compute_metrics(start_logits, end_logits, processed_dataset, orig_dataset):

    n_largest = 1
    max_answer_length = 30

    sample_id2idxs = defaultdict(list)

    for i, sample in enumerate(processed_dataset):
        sample_id = sample['sample_id']
        sample_id2idxs[sample_id].append(i)

    predicted_answers = []

    for sample in tqdm(orig_dataset):
        sample_id = sample['id']
        context = sample['context']
        best_score = float('-inf')
        best_answer = None

        for idx in sample_id2idxs[sample_id]:
            start_logit = start_logits[idx]
            end_logit = end_logits[idx]

            offsets = processed_dataset[idx]['offset_mapping']

            start_indices = (-start_logit).argsort()[:n_largest]
            end_indices = (-end_logit).argsort()[:n_largest]

            for start_idx in start_indices:
                for end_idx in end_indices:
                    if offsets[start_idx] is None or offsets[end_idx] is None:
                        continue

                    if end_idx < start_idx or (end_idx - start_idx + 1) > max_answer_length:
                        continue

                    score = start_logit[start_idx] + end_logit[end_idx]

                    if score > best_score:
                        best_score = score
                        first_ch = offsets[start_idx][0]
                        last_ch = offsets[end_idx][1]
                        best_answer = context[first_ch:last_ch]

        predicted_answers.append({'id': sample_id, 'prediction_text': best_answer})

    true_answers = [{'id': x['id'], 'answers': x['answers']} for x in orig_dataset]

    metric_score = metric.compute(predictions=predicted_answers, references=true_answers)

    return metric_score


In [48]:
compute_metrics(start_logits,
                end_logits,
                small_validation_processed,
                small_validation_dataset)

100%|██████████| 100/100 [00:00<00:00, 790.71it/s]


{'exact_match': 83.0, 'f1': 88.25000000000004}

In [49]:
model=AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
from transformers import TrainingArguments
args=TrainingArguments(
    'finetuned-squad',
    evaluation_strategy="epoch",
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=False,
    fp16_full_eval=False
)

In [58]:
#since it took a lot time training is paused
from transformers import Trainer
trainer=Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
    )
trainer.train()

In [ ]:
trainer_output=trainer.predict(validation_dataset)

In [ ]:
type(trainer_output)

''

In [ ]:
trainer_output

''

In [ ]:
predictions,_,_=trainer_output

''

In [ ]:
predictions

In [ ]:
start_logits,end_logits=predictions

In [ ]:
compute_metrics=(start_logits,end_logits,validation_dataset,raw_datasets['validation'])

In [ ]:
trainer.save_model('my_saved_model')

In [ ]:
from transformers import pipeline

In [ ]:
qa=pipeline(
    'question-answering',
    model='my_saved_model',
    device=0
)

In [ ]:
context='Today I went to the store to purchase a cartoon of milk.'
question='What did I buy'
qa(context=context,question=question)